<a href="https://colab.research.google.com/github/EdoardoCarlesi/notebooks/blob/main/BorsinoAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycurl

In [2]:
import pandas as pd
import requests
import json
from requests.auth import HTTPBasicAuth

In [3]:
# API libere, accesso senza key
api_url_immobili='https://api.borsinopro.it/rest/standard-v1/getImmobiliType/'
api_url_contratti='https://api.borsinopro.it/rest/standard-v1/getContractType/'

# API pro, accesso con key
api_url_trends='https://api.borsinopro.it/rest/professional-v2/getTrends/'
api_url_demografia='https://api.borsinopro.it/rest/professional-v2/getDemographic/'
api_url_tempistiche='https://api.borsinopro.it/rest/professional-v2/getTempistiche/'

# API key temporanea
api_key='ZWRvYXJkby5jYXJsZXNpQHRvcC1uZXR3b3JrLml0OmRlbW8='



In [4]:
# Tipo di contratto

contratto = requests.get(api_url_contratti)
print(contratto.status_code)
contratto.json()['response']['options']

200


{'rent': {'id': 'rent', 'label': 'Affitto'},
 'sale': {'id': 'sale', 'label': 'Vendita'}}

In [5]:
# Tipo di immobili

immobili = requests.get(api_url_immobili)
immobili.json()['response']['options']

[{'id': '20', 'label': 'Abitazioni in stabili civili'},
 {'id': '19', 'label': 'Abitazioni in stabili signorili'},
 {'id': '21', 'label': 'Abitazioni in stabili economici'},
 {'id': '1', 'label': 'Ville e Villini'},
 {'id': '6', 'label': 'Uffici'},
 {'id': '5', 'label': 'Negozi'},
 {'id': '13', 'label': 'Box e Autorimesse'},
 {'id': '14', 'label': 'Posto auto coperti'},
 {'id': '15', 'label': 'Posto auto scoperti'},
 {'id': '9', 'label': 'Magazzini'},
 {'id': '10', 'label': 'Laboratori'},
 {'id': '7', 'label': 'Capannoni tipici'},
 {'id': '8', 'label': 'Capannoni industriali'}]

In [6]:
# Call the getTrends() API with API key

tipo_immobile=immobili.json()['response']['options'][0]['id']
print(tipo_immobile)
tipo_contratto=contratto.json()['response']['options']['rent']['label']
print(tipo_contratto)

# Set the parameters for the request
indirizzo="Via del Corso Roma"
header={'Authorization':'Basic '+api_key, 'Content-Type': 'application/json'}
method='post'
#data_post=json.dumps({"address":indirizzo, "type":tipo_immobile, "for":tipo_contratto})
#data_post=json.dumps({"address": indirizzo, "type": tipo_immobile, "for": tipo_contratto})
data_post={"address": indirizzo, "type": tipo_immobile, "for": tipo_contratto}

# requests cannot handle properly the calls to these APIs
#print(data_post['address'])
#print(data_post)
#rsp=requests.request(method, api_url_trends, params=data_post, headers=header)
#rsp=requests.post(api_url_trends, data=data_post, headers=header)
#rsp=requests.request(method, api_url_trends, json=data_post, headers=header)
#rsp=requests.request(method, api_url_trends, data=data_post, headers=header)
#rsp.json()

20
Affitto


**Using PyCurl for the pro-API**

La libreria requests non permette di formattare i dati in ingresso per la richiesta nel formato accettato da Borsino (che non e' JSON), per cui per utilizzare le API pro con una POST request e' necessario 

In [7]:
import pycurl
from urllib.parse import urlencode
from io import BytesIO

data_curl = urlencode(data_post)
headers=['Authorization: Basic '+api_key]

def getTrends(api_url=None, api_key=None, headers=None, data=None):
  '''
  '''  

  crl = pycurl.Curl()
  response = BytesIO()
  crl.setopt(crl.URL, api_url_trends)
  crl.setopt(crl.HTTPAUTH, crl.HTTPAUTH_BASIC)
  crl.setopt(crl.USERPWD, api_key)
  crl.setopt(crl.HTTPHEADER, headers)
  crl.setopt(crl.CUSTOMREQUEST, 'POST')
  crl.setopt(crl.POSTFIELDS, data)
  crl.setopt(crl.WRITEFUNCTION, response.write)
  crl.perform()
  rsp_dict = json.loads(response.getvalue().decode("utf-8"))
  
  return rsp_dict

address=Via+del+Corso+Roma&type=20&for=Affitto
{"address": "Via del Corso Roma", "type": "20", "for": "Affitto"}
b'{"method":"getTrends","response":{"id":"17671","nome":"Centro Storico Tridente- Campo Marzio, Colonna, Pigna, Trevi","id_citta":"2167","citta":"Roma","cod_cat":"H501","provincia":"RM","id_provincia":"86","id_regione":"7","regione":"Lazio","fascia":"centrale","address":"Via del Corso Roma","formatted":"Via del Corso, Roma RM, Italia","loc2011":"580911","coordinate":{"lat":41.90248,"lng":12.47982},"immobile":{"type":"Abitazioni in stabili civili","label":"Valori di Vendita Abitazioni in stabili civili al Mq","id":20},"quotazione":{"sale_min":"5426","sale_diff":"2841","sale_max":"8267","rent_min":"17.03","rent_diff":"8.91","rent_max":"25.94","type":"sale","med":0,"min":0,"max":0},"ripartizione_annunci":{"vendita":{"1":{"tipologia":"Appartamenti","percentuale":63.09},"2":{"tipologia":"Locali ed Uffici","percentuale":29.54},"12":{"tipologia":"Ville e Villini","percentuale":3.36

In [9]:
# Questa sintassi funziona solo con python 3.8+
#print(sale_min := rsp_dict['response']['quotazione']['sale_min'])
#print(rent_min := rsp_dict['response']['quotazione']['rent_min'])

rsp_dict = getTrends(api_url=api_url_trends, api_key=api_key, headers=headers, data=data_curl)
sale_min = rsp_dict['response']['quotazione']['sale_min']
rent_min = rsp_dict['response']['quotazione']['rent_min']
codice = rsp_dict['univoco']

print(codice)
print(f'Prezzo Vendita Minimo:{sale_min}, Prezzo Affitto Minimo: {rent_min}')

20210916144124_61433b74f393b_20_3649
Prezzo Vendita Minimo:5426, Prezzo Affitto Minimo: 17.03


In [11]:
# Get Demographic
data_post={'univoco':codice}
data_curl = urlencode(data_post)
headers=['Authorization: Basic '+api_key]

def getDemographic(api_url=None, api_key=None, headers=None, data=None):
  '''
  '''

  crl = pycurl.Curl()
  response_demo = BytesIO()
  crl.setopt(crl.URL, api_url)
  crl.setopt(crl.HTTPAUTH, crl.HTTPAUTH_BASIC)
  crl.setopt(crl.USERPWD, api_key)
  crl.setopt(crl.HTTPHEADER, headers)
  crl.setopt(crl.CUSTOMREQUEST, 'POST')
  crl.setopt(crl.POSTFIELDS, data)
  crl.setopt(crl.WRITEFUNCTION, response_demo.write)
  crl.perform()
  rsp_demo_dict = json.loads(response_demo.getvalue().decode("utf-8"))
 
  return rsp_demo_dict
  

{'method': 'getDemographic',
 'response': {'capoluogo': [{'dinamicita_mercato_residenziale': 'medio bassa',
    'media_num_abitazioni_zona': '8760.4000',
    'media_numero_abitazioni_vendute': '109.8000'}],
  'geo': {'altitudine': '0',
   'cod_cat': 'H501',
   'id': '117',
   'id_provincia': '86',
   'loc2011': '580911',
   'nome_localita': 'I Municipio',
   'tipo_localita': 'Municipio'},
  'mappa': 'https://api.borsinopro.it/mappa/20210916144124_61433b74f393b_20_3649/',
  'pop': {'dimensione_nuclei_familiari': 'piccoli',
   'livello_anzianita': 'medio',
   'livello_disoccupazione': 'medio bassa',
   'livello_istruzione': 'superiore alla media',
   'media_componenti_famiglia': '1.68'},
  'rischio_sismico': 'inferiore alla media',
  'stock': {'abitanti_per_edificio': '19.00',
   'abitazioni_per_edificio': '10.44',
   'mq_abitazione_media': '96.80',
   'prc_abitazioni_di_proprieta': '60.000',
   'prc_abitazioni_vuote_o_2_case': '21.000',
   'prc_edifici_produttivi_commerciali': '36.000',

In [13]:
# Get tempistiche
def getTime():
  '''
  '''

  crl = pycurl.Curl()
  response_temp = BytesIO()
  crl.setopt(crl.URL, api_url_demografia)
  crl.setopt(crl.HTTPAUTH, crl.HTTPAUTH_BASIC)
  crl.setopt(crl.USERPWD, api_key)
  crl.setopt(crl.HTTPHEADER, headers)
  crl.setopt(crl.CUSTOMREQUEST, 'POST')
  crl.setopt(crl.POSTFIELDS, data_curl)
  crl.setopt(crl.WRITEFUNCTION, response_temp.write)
  crl.perform()
  rsp_temp_dict = json.loads(response_temp.getvalue().decode("utf-8"))

  return rsp_temp_dict

**Poligoni shapefiles per le zone dell'OMI**

File completo

http://dev.ondata.it/projs/zoneomi/zone_omi_poligoni.zip


Altri links utili:
http://blog.ondata.it/i-dati-sulle-quotazioni-immobiliari-dellagenzia-entrate-i-poligoni-delle-zone-omi/

https://www.istat.it/it/archivio/222527

https://www.provinceditalia.it/mappaprovince/

https://www1.agenziaentrate.gov.it/servizi/geopoi_omi/index.php

In [ ]:
!pip install geopandas geopy utm


In [ ]:
%cd MyDrive
#!wget https://github.com/ondata/quotazioni-immobiliari-agenzia-entrate/blob/master/rawData/2016-2.zip
!wget https://github.com/ondata/quotazioni-immobiliari-agenzia-entrate/archive/refs/heads/master.zip

In [56]:
%cd /content/drive/MyDrive/
#!unzip master.zip
#!unzip /content/drive/MyDrive/quotazioni-immobiliari-agenzia-entrate-master/rawData/2018-2.zip

/content/drive/MyDrive


In [77]:
import geopandas as gpd

# Zone Omi
omi_path='/content/drive/MyDrive/zone_omi/'
omi_shp=omi_path+'zone_omi_all.shp'

gdf = gpd.read_file(omi_shp)
#print(gdf.head())
#gdf.head(20)
gdf_rm = gdf[gdf['Comune'] == 'Roma']
gdf_sc = gdf[gdf['QI_29457_1'] == 'SICILIA']
print(len(gdf_rm), len(gdf_sc))
gdf_rm.head()
#print(len(gdf))

234 1644


,Name,CODCOM,CODZONA,QI_294577_,QI_29457_1,QI_29457_2,QI_29457_3,Com1991_WG,Com1991__1,Com1991__2,Com1991__3,Com1991__4,Comune,Com1991__6,Shape_area,Shape_leng,geometry
7196,ROMA - Zona OMI D8,H501,D8,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,3.257845e+06,13551.618000,"POLYGON Z ((783972.048 4636648.953 0.000, 7839..."
7197,ROMA - Zona OMI D53,H501,D53,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,1.402869e+06,7820.129817,"POLYGON Z ((793818.479 4641566.688 0.000, 7938..."
7198,ROMA - Zona OMI D78,H501,D78,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,9.032285e+05,5065.345244,"POLYGON Z ((787478.768 4634958.856 0.000, 7875..."
7199,ROMA - Zona OMI D24,H501,D24,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,3.811554e+06,11452.173834,"POLYGON Z ((781938.020 4645444.715 0.000, 7819..."
7200,ROMA - Zona OMI D17,H501,D17,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,1.366475e+06,5548.683341,MULTIPOLYGON Z (((794293.295 4644839.086 0.000...


In [34]:
# Let's find the center of each polygon

geometries = gdf_rm['geometry']
centers = []

for geo in geometries:
  centers.append(geo.centroid)

gdf_rm['centers'] = centers
gdf_rm.head()

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,Name,CODCOM,CODZONA,QI_294577_,QI_29457_1,QI_29457_2,QI_29457_3,Com1991_WG,Com1991__1,Com1991__2,Com1991__3,Com1991__4,Comune,Com1991__6,Shape_area,Shape_leng,geometry,centers
7196,ROMA - Zona OMI D8,H501,D8,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,3.257845e+06,13551.618000,"POLYGON Z ((783972.048 4636648.953 0.000, 7839...",POINT (786513.5019016892 4637762.815119482)
7197,ROMA - Zona OMI D53,H501,D53,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,1.402869e+06,7820.129817,"POLYGON Z ((793818.479 4641566.688 0.000, 7938...",POINT (794559.3531067014 4640666.958689639)
7198,ROMA - Zona OMI D78,H501,D78,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,9.032285e+05,5065.345244,"POLYGON Z ((787478.768 4634958.856 0.000, 7875...",POINT (788330.8834137139 4635315.78231564)
7199,ROMA - Zona OMI D24,H501,D24,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,3.811554e+06,11452.173834,"POLYGON Z ((781938.020 4645444.715 0.000, 7819...",POINT (783287.8355003501 4646733.881924237)
7200,ROMA - Zona OMI D17,H501,D17,CENTRO,LAZIO,RM,12058091.0,3.0,12.0,58.0,58091.0,058091,Roma,None,1.366475e+06,5548.683341,MULTIPOLYGON Z (((794293.295 4644839.086 0.000...,POINT (795180.828549902 4644708.600154188)


In [ ]:
omi_values='/content/drive/MyDrive/QI_294577_1_20182_VALORI.csv'
omi_zones='/content/drive/MyDrive/QI_294577_1_20182_ZONE.csv'

df_val = gpd.read_file(omi_zones)
df_val.head()

In [102]:
# Do the reverse geocoding operation - for each center of the list above get 
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# EXTREMELY IMPORTANT!!!!!
import utm 

locator = Nominatim(user_agent='MyGeocoder')

'''
THIS DOES NOT WORK WITH THE KIND OF UTM COORDINATES AVAILABLE
from pyproj import Proj, transform
#in_coords = Proj(init='epsg:3031')
in_coords = Proj(init='epsg:163716')
out_coords = Proj(init='epsg:4326')
coords = transform(in_coords, out_coords, centers[0].x, centers[0].y)
'''

# TODO! This is ok for Italy but needs to be checked for ALL the provinces...
zone_number = 32

for center in centers[0:10]:
  coords = utm.to_latlon(center.x, center.y, zone_number, 'U')
  location = locator.reverse(coords)
  print(location.raw['address'])

{'road': 'Largo Giuseppe Petrelli', 'suburb': 'Suburbio VII Portuense', 'city': 'Roma', 'county': 'Roma Capitale', 'state': 'Lazio', 'postcode': '00144', 'country': 'Italia', 'country_code': 'it'}
{'road': 'Via dei Furi', 'neighbourhood': 'Quadraretto', 'suburb': 'Quartiere VIII Tuscolano', 'city': 'Roma', 'county': 'Roma Capitale', 'state': 'Lazio', 'postcode': '00174', 'country': 'Italia', 'country_code': 'it'}
{'shop': 'Carrefour Express', 'house_number': '284', 'road': 'Viale Cesare Pavese', 'suburb': 'Zona XXIV Fonte Ostiense', 'city': 'Roma', 'county': 'Roma Capitale', 'state': 'Lazio', 'postcode': '00144', 'country': 'Italia', 'country_code': 'it'}
{'amenity': 'Galaxi', 'road': 'Via Pietro Maffi', 'suburb': 'Municipio Roma XIV', 'city': 'Roma', 'county': 'Roma Capitale', 'state': 'Lazio', 'postcode': '00167', 'country': 'Italia', 'country_code': 'it'}
{'road': 'Viale della Serenissima', 'suburb': 'Municipio Roma V', 'city': 'Roma', 'county': 'Roma Capitale', 'state': 'Lazio', 'p